# optionm

In [ ]:
//// test

open testing

()



In [ ]:
open rust_operators

()



## default_value

In [ ]:
inl default_value d =
    optionm.defaultWith d

()



In [ ]:
//// test

None
|> default_value 3i32
|> _assert_eq 3i32

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{3} / expected: %A{3}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 3 / expected: 3


## (/??)

In [ ]:
inl (/??) a b =
    a |> default_value b

()



In [ ]:
//// test

None /?? 3i32
|> _assert_eq 3i32

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{3} / expected: %A{3}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 3 / expected: 3


## default_with

In [ ]:
inl default_with fn = function
    | Some x => x
    | None => fn ()

()



In [ ]:
//// test

None
|> default_with fun () => 3i32
|> _assert_eq 3i32

let rec method0 () : unit =
    let v0 : string = "assert_eq"
    let v1 : string = $"{v0} / actual: %A{3} / expected: %A{3}"
    System.Console.WriteLine v1
    ()
method0()

assert_eq / actual: 3 / expected: 3


## choose

In [ ]:
inl choose fn a b =
    match a, b with
    | Some x, Some y => fn x y |> Some
    | _ => None

()



In [ ]:
//// test

(Some 2i32, Some 3)
||> choose (+)
|> _assert_eq (Some 5)

type [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
let rec method0 () : unit =
    let v3 : string = "assert_eq"
    let v4 : US0 = US0_0(5)
    let v5 : US0 = US0_0(5)
    let v6 : string = $"{v3} / actual: %A{v4} / expected: %A{v5}"
    System.Console.WriteLine v6
    ()
method0()

assert_eq / actual: US0_0 5 / expected: US0_0 5


## iter

In [ ]:
inl iter fn = function
    | Some x => fn x
    | None => ()

()



In [ ]:
//// test

inl n = mut 1i32
inl fn =
    fun n' =>
        n <- *n + n'
Some 1i32 |> iter fn
None |> iter fn
*n
|> _assert_eq 2i32

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 1} : Mut0
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    let v3 : int32 = v0.l0
    let v4 : bool = v3 = 2
    let v6 : bool =
        if v4 then
            true
        else
            method1(v4)
    let v7 : string = "assert_eq"
    let v8 : string = $"{v7} / actual: %A{v3} / expected: %A{2}"
    System.Console.WriteLine v8
    let v9 : bool = v6 = false
    if v9 then
        failwith<unit> v8
method0()

assert_eq / actual: 2 / expected: 2


## option'

In [ ]:
nominal option' t = $"`t option"

()



## none'

In [ ]:
inl none' forall t. () : option' t =
    $'None'

()



## some'

In [ ]:
inl some' forall t. (x : t) : option' t =
    $'Some !x '

()



## default_value'

In [ ]:
inl default_value' forall t. (value : t) (x : option' t) : t =
    $'!x |> Option.defaultValue !value '

()



## try'

In [ ]:
inl try' forall t. (x : option' t) : t =
    !\\(x, $'"$0?"')

()



## map

In [ ]:
inl map forall t u. (fn : t -> u) (x : option' t) : option' u =
    inl result : option' u = none' ()
    $'let _!result = ref !result '
    $'match !x with'
    $'| Some x -> ('
    $'(fun () ->'
    $'(fun () ->'
    inl x = dyn $'x'
    x |> fn |> emit_unit
    $')'
    $"|> fun x -> x () |> Some"
    $') () ) | None -> None'
    $"|> fun x -> _!result.Value <- x"
    $'_!result.Value '

()



## box

In [ ]:
inl box forall t. (x : option t) : option' t =
    // x
    // |> optionm.map some'
    // |> default_with none'
    match x with
    | Some x => some' x
    | None => none' ()

()



## map'

In [ ]:
inl map' forall t u. (fn : t -> u) (x : option' t) : option' u =
    (!\($'"true; let _result = !x.map(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result"')

()



## map''

In [ ]:
inl map'' forall t u. (fn : t -> u) (x : option' t) : option' u =
    $'!x |> Option.map !fn '

()



## unbox

In [ ]:
inl unbox forall t. (x : option' t) : option t =
    x |> map Some |> default_value' None
    // inl some x : option t = Some x
    // inl some = join some
    // inl none : option t = None
    // $'!x |> Option.map !some |> Option.defaultValue !none '

()



In [ ]:
//// test

inl x = Some 3i32
inl y : option i32 = None
inl x' = x |> box |> unbox
inl y' = y |> box |> map id |> unbox
(x', y') |> _assert_eq (x, y)

type [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 option = Some 3 
    let v1 : US0 option = None
    let _v1 = ref v1 
    match v0 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v2 : int32 = x
    let v3 : US0 = US0_0(v2)
    v3 
    )
    |> fun x -> x () |> Some
    ) () ) | None -> None
    |> fun x -> _v1.Value <- x
    let v4 : US0 option = _v1.Value 
    let v5 : US0 = US0_1
    let v6 : US0 = v4 |> Option.defaultValue v5 
    let v7 : int32 option = None
    let v8 : int32 option = None
    let _v8 = ref v8 
    match v7 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v9 : int32 = x
    v9 
    )
    |> fun x -> x () |> Some
    ) () ) | None -> None
    |> fun x -> _v8.Value <- x
    let v10 : int32 option = _v8.Value 
    let v11 : US0 option = None
    let _v11 = ref v11 
    match v10 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v12 

In [ ]:
//// test
//// print_code=true
///! rust

inl x = Some 3i32
inl y : option i32 = None
inl x' = x |> box |> unbox
inl y' = y |> box |> map id |> unbox
(x', y') |> _assert_eq' (x, y)

.rs output:
assert_eq' / actual: (US0_0(3), US0_1) / expected: (US0_0(3), US0_1)




.fsx:
type [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 option = Some 3 
    let v1 : US0 option = None
    let _v1 = ref v1 
    match v0 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v2 : int32 = x
    let v3 : US0 = US0_0(v2)
    v3 
    )
    |> fun x -> x () |> Some
    ) () ) | None -> None
    |> fun x -> _v1.Value <- x
    let v4 : US0 option = _v1.Value 
    let v5 : US0 = US0_1
    let v6 : US0 = v4 |> Option.defaultValue v5 
    let v7 : int32 option = None
    let v8 : int32 option = None
    let _v8 = ref v8 
    match v7 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v9 : int32 = x
    v9 
    )
    |> fun x -> x () |> Some
    ) () ) | None -> None
    |> fun x -> _v8.Value <- x
    let v10 : int32 option = _v8.Value 
    let v11 : US0 option = None
    let _v11 = ref v11 
    match v10 with
    | Some x -> (
    (fun () ->
    (fun () ->
    le

## of_obj

In [ ]:
inl of_obj forall t. (x : t) : option' t =
    $'let mutable _!x = None'
    $"#if \!FABLE_COMPILER && \!WASM && \!CONTRACT"
    ((x |> $'Option.ofObj') : option' t)
    |> fun x => $'!x '
    $"#else"
    $'Some !x '
    $"#endif"
    $"|> fun x -> _!x <- Some x"
    $'_!x.Value'

()



In [ ]:
//// test

null ()
|> of_obj
|> unbox
|> _assert_eq (None : option string)

type [<Struct>] US0 =
    | US0_0 of f0_0 : string
    | US0_1
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = null |> unbox<string>
    let mutable _v0 = None
    #if !FABLE_COMPILER && !WASM && !CONTRACT
    let v1 : (string -> string option) = Option.ofObj
    let v2 : string option = v1 v0
    v2 
    #else
    Some v0 
    #endif
    |> fun x -> _v0 <- Some x
    let v3 : string option = _v0.Value
    let v4 : US0 option = None
    let _v4 = ref v4 
    match v3 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v5 : string = x
    let v6 : US0 = US0_0(v5)
    v6 
    )
    |> fun x -> x () |> Some
    ) () ) | None -> None
    |> fun x -> _v4.Value <- x
    let v7 : US0 option = _v4.Value 
    let v8 : US0 = US0_1
    let v9 : US0 = v7 |> Option.defaultValue v8 
    let v11 : bool =
        match v9 with
        | US0_1 -> (* None *)
            true
        | _ ->
            false
    let v13 : bool =
        if v11 then
     

In [ ]:
//// test
///! rust

""
|> of_obj
|> unbox
|> _assert_eq' (Some "")

.rs output:
assert_eq' / actual: US0_0("") / expected: US0_0("")




.fsx:
type [<Struct>] US0 =
    | US0_0 of f0_0 : string
    | US0_1
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = ""
    let mutable _v0 = None
    #if !FABLE_COMPILER && !WASM && !CONTRACT
    let v1 : (string -> string option) = Option.ofObj
    let v2 : string option = v1 v0
    v2 
    #else
    Some v0 
    #endif
    |> fun x -> _v0 <- Some x
    let v3 : string option = _v0.Value
    let v4 : US0 option = None
    let _v4 = ref v4 
    match v3 with
    | Some x -> (
    (fun () ->
    (fun () ->
    let v5 : string = x
    let v6 : US0 = US0_0(v5)
    v6 
    )
    |> fun x -> x () |> Some
    ) () ) | None -> None
    |> fun x -> _v4.Value <- x
    let v7 : US0 option = _v4.Value 
    let v8 : US0 = US0_1
    let v9 : US0 = v7 |> Option.defaultValue v8 
    let v10 : US0 = US0_0(v0)
    let v11 : bool = v9 = v10
    let v13 : bool =
        if v11 then
            true
        else
            method1(v11)
    let v14 : string = "as

## flatten

In [ ]:
inl flatten x =
    match x with
    | Some (Some x) => Some x
    | _ => None

()



## unwrap

In [ ]:
inl unwrap forall t. (x : option' t) : t =
    !\\(x, $'"$0.unwrap()"')

()



## unwrap_or

In [ ]:
inl unwrap_or forall t. (def : t) (x : option' t) : t =
    !\($'"!x.unwrap_or(!def)"')

()



## rc_upgrade

In [ ]:
inl rc_upgrade forall t. (x : rust.weak_rc t) : option' (rust.rc t) =
    !\\(x, $'"std::rc::Weak::upgrade(&$0)"')

()



## rc_into_inner

In [ ]:
inl rc_into_inner forall t. (x : rust.rc t) : option' t =
    !\\(x, $'"std::rc::Rc::into_inner($0)"')

()



In [ ]:
//// test
//// print_code=false
///! rust

types ()
rust.new_rc 0i32
|> rc_into_inner
|> unbox
|> _assert_eq' (Some 0i32)

assert_eq' / actual: US0_0(0) / expected: US0_0(0)
